In [3]:
import os 
from dotenv import load_dotenv

In [4]:
load_dotenv()
HUGGINFACE_API_KEY = os.getenv("HUGGINFACE_API_KEY")
#HUGGINFACE_API_KEY

In [5]:
os.environ["HUGGINFACE_API_KEY"] = HUGGINFACE_API_KEY

In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [7]:
%pwd

'c:\\Users\\sanna\\OneDrive\\Desktop\\GenAI\\Interview_QA\\research'

In [8]:
%cd ..


c:\Users\sanna\OneDrive\Desktop\GenAI\Interview_QA


c:\Users\sanna\conda\anaconda3\envs\interview\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
file_path = "data/SDG.pdf"
loader = PyPDFLoader(file_path)
data=loader.load()
data


[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 2, 'page_label': '3'}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE T

In [10]:
len(data)

24

In [11]:
#load only the page content to one variable
question_gen = ""
for page in data:
    question_gen += page.page_content
question_gen

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 yea

In [12]:
# create chunks
from langchain_text_splitters import TokenTextSplitter
from transformers import AutoTokenizer

#use llm for the context remember instead of chunk_overlap
hf_model = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(hf_model)

splitter_ques_gen = TokenTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size = 10000,
    chunk_overlap = 200
)
 

In [13]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [14]:
#only consider all the data to 1 chunk
len(chunk_ques_gen)

1

In [15]:
#chunk is a string fromat we need document to work on , string will also work but document format is required way
from langchain.docstore.document import Document

documet_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

In [16]:
documet_ques_gen
type(documet_ques_gen[0])

langchain_core.documents.base.Document

In [17]:
# do the chunking prev was a demo 
splitter_ans_gen = TokenTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size = 1000,
    chunk_overlap = 100
)

In [18]:
documet_answer_gen = splitter_ans_gen.split_documents(
    documet_ques_gen
)

In [19]:
len(documet_answer_gen)

5

In [20]:
# generate the interview question - Initialise the LLM
from langchain.llms import HuggingFaceEndpoint


hf_model = "mistralai/Mistral-7B-Instruct-v0.1"

llm = HuggingFaceEndpoint(
    endpoint_url=f"https://api-inference.huggingface.co/models/{hf_model}",
    huggingfacehub_api_token=HUGGINFACE_API_KEY,
    temperature=0.3,  # Controls randomness
    model_kwargs={"max_length": 512}
)

C:\Users\sanna\AppData\Local\Temp\ipykernel_8512\3654105830.py:7: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(


In [21]:
prompt_template = """
You are an expert in Sustainable Development Goals (SDGs) and global sustainability initiatives.  
Your goal is to help individuals, students, and professionals deepen their understanding of SDGs  
by generating insightful and thought-provoking questions based on the text below:  

------------  
{text}  
------------  

Create questions that will enhance understanding and encourage critical thinking about the SDGs.  
Ensure that the questions cover key concepts, challenges, and solutions related to sustainable development.  

QUESTIONS:
"""


In [22]:
from langchain.prompts import PromptTemplate 

PROMPT_QUESTIONS = PromptTemplate(template = prompt_template , input_variables = ["text"])

In [23]:
# We can use a refine template to make the answer better , its by giving the prev generated questions to a new refined prompt template to check if there is any irrevelant questions avaiable 

refine_template = ("""
You are an expert in sustainable development and global initiatives.  
Your goal is to provide well-informed, accurate, and insightful answers related to the United Nations’ Sustainable Development Goals (SDGs).  

We have received an initial answer: {existing_answer}.  
We have the option to refine this response or provide additional details if necessary, based on the context below.  

------------  
{text}  
------------  

Given the new context, refine or expand the original response in English.  
If the context does not add relevant information, please provide the original response.  

ANSWER:  
"""
)


In [24]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [25]:
#connect the llm and our prompt_template

from langchain.chains.summarize import load_summarize_chain

ques_gen_chain =  load_summarize_chain(llm = llm,
                                       chain_type = "refine",#its a refine prompt
                                       verbose = True,
                                       question_prompt = PROMPT_QUESTIONS,
                                       refine_prompt = REFINE_PROMPT_QUESTIONS)

In [26]:
# now pass the document of chunks to the chain of llm

ques = ques_gen_chain.run(documet_ques_gen)

print(ques)

C:\Users\sanna\AppData\Local\Temp\ipykernel_8512\3599558461.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques = ques_gen_chain.run(documet_ques_gen)
c:\Users\sanna\conda\anaconda3\envs\interview\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert in Sustainable Development Goals (SDGs) and global sustainability initiatives.  
Your goal is to help individuals, students, and professionals deepen their understanding of SDGs  
by generating insightful and thought-provoking questions based on the text below:  

------------  
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough 
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew 
that earthquakes and floods were inevitable, but that the high death 
tolls were not. 
They also knew that billions of people

In [34]:
# now create a embedding model to pass the question genearted for getting the answer generated 
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINFACE_API_KEY, model_name="sentence-transformers/all-MiniLM-L6-v2"
)



In [35]:
test_text = "What are the key challenges in achieving SDG 13?"
test_embedding = embeddings.embed_query(test_text)

print(len(test_embedding))  # Should be a number (e.g., 384 for MiniLM)
print(test_embedding[:5]) 

384
[-0.04814745858311653, 0.04864189028739929, 0.05261829495429993, -0.07543892413377762, -0.016091857105493546]


In [36]:
from langchain_community.vectorstores import FAISS

In [37]:
# initiliase the fiass vector store

vectore_store = FAISS.from_documents(documet_answer_gen, embeddings)


In [50]:
#now use Retrival QA chain to pass vector to llm

from langchain.chains import RetrievalQA

answer_generation_chain = RetrievalQA.from_chain_type(llm=llm,
                                                chain_type = "stuff",
                                                retriever=vectore_store.as_retriever())

In [46]:
ques

'\n1. What are the SDGs and why are they important?\n2. What are some of the main challenges facing the world today?\n3. How can we end extreme poverty and hunger?\n4. What is the importance of good health and well-being for all?\n5. How can we ensure inclusive and equitable quality education and promote lifelong learning opportunities for all?\n6. What are some of the key factors that contribute to gender inequality and discrimination against women and girls?\n7. How can we ensure access to safe and affordable drinking water for all?\n8. What are some of the key factors that contribute to climate change and how can we address them?\n9. How can we promote sustained, inclusive, and sustainable economic growth, full and productive employment, and decent work for all?\n10. What are some of the key factors that contribute to resilient infrastructure, inclusive and sustainable industrialization, and fostering innovation?\n11. How can we reduce inequality within and among countries?\n12. Wha

In [47]:
ques_list = ques.split("\n")
ques_list

['',
 '1. What are the SDGs and why are they important?',
 '2. What are some of the main challenges facing the world today?',
 '3. How can we end extreme poverty and hunger?',
 '4. What is the importance of good health and well-being for all?',
 '5. How can we ensure inclusive and equitable quality education and promote lifelong learning opportunities for all?',
 '6. What are some of the key factors that contribute to gender inequality and discrimination against women and girls?',
 '7. How can we ensure access to safe and affordable drinking water for all?',
 '8. What are some of the key factors that contribute to climate change and how can we address them?',
 '9. How can we promote sustained, inclusive, and sustainable economic growth, full and productive employment, and decent work for all?',
 '10. What are some of the key factors that contribute to resilient infrastructure, inclusive and sustainable industrialization, and fostering innovation?',
 '11. How can we reduce inequality wi

In [51]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")




Question:  


c:\Users\sanna\conda\anaconda3\envs\interview\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Answer:   

The Sustainable Development Goals (SDGs) are a set of 17 goals that were adopted by the United Nations in 2015. These goals aim to end poverty, hunger, and disease, and to promote sustainable development by 2030. The goals are interconnected and require a global partnership to be achieved. To implement the SDGs, the United Nations Development Programme (UNDP) is working with countries to identify and address the challenges they face in achieving the goals. The UNDP is also advocating for increased funding and support for the SDGs, and encouraging individuals and organizations to take action to make a difference.
--------------------------------------------------\n\n
Question:  1. What are the SDGs and why are they important?


c:\Users\sanna\conda\anaconda3\envs\interview\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Answer:   The SDGs are a set of 17 goals adopted by the United Nations in 2015 to end poverty, protect the planet and ensure that all people enjoy peace and prosperity. They are important because they provide a universal call to action to end poverty, protect the planet and ensure that all people enjoy peace and prosperity. The goals cover a wide range of issues, from eliminating extreme poverty and hunger to promoting sustainable development, combating climate change and building resilient infrastructure. Achieving the SDGs will require the collective efforts of governments, businesses and civil society, and will require innovative solutions and new ways of thinking.
--------------------------------------------------\n\n
Question:  2. What are some of the main challenges facing the world today?


...

c:\Users\sanna\conda\anaconda3\envs\interview\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Answer:   8. Climate change is caused by a variety of factors, including the burning of fossil fuels, deforestation, and industrial processes. To address climate change, we need to reduce greenhouse gas emissions, shift to renewable energy sources, and promote sustainable land use practices. This can be done through policies and actions such as carbon pricing, investing in renewable energy, and promoting reforestation and sustainable agriculture. Additionally, we need to promote international cooperation and collaboration to address climate change on a global scale.
--------------------------------------------------\n\n
Question:  9. How can we promote sustained, inclusive, and sustainable economic growth, full and productive employment, and decent work for all?


c:\Users\sanna\conda\anaconda3\envs\interview\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Answer:   We can promote policies that encourage entrepreneurship and job creation. We can eradicate forced labour, slavery and human trafficking. And in the end we can achieve the goal of decent work for all women and men by 2030.
--------------------------------------------------\n\n
Question:  10. What are some of the key factors that contribute to resilient infrastructure, inclusive and sustainable industrialization, and fostering innovation?


KeyboardInterrupt: 